In [1]:
import os 
import cv2
import pandas as pd
import numpy as np 
from tqdm import tqdm
import Functions as func
import var

Using plaidml.keras.backend backend.


The general flow of our algorithm will be:

1. **Step #1:** Input an image
2. **Step #2:** Construct an image pyramid
3. **Step #3:** For each scale of the image pyramid, run a sliding window
4. **Step #3a:** For each stop of the sliding window, extract the ROI
5. **Step #3b:** Take the ROI and pass it through our CNN originally trained for image classification
6. **Step #3c:** Examine the probability of the top class label of the CNN, and if meets a minimum confidence, record (1) the class label and (2) the location of the sliding window
7. **Step #4:** Apply class-wise non-maxima suppression to the bounding boxes
8. **Step #5:** Return results to calling function

In [2]:
def get_iou(box1, box2):
    # determine the (x, y)-coordinates of the intersection rectangle
    xa = max(box1[0], box2[0])
    ya = max(box1[1], box2[1])
    xb = min(box1[2], box2[2])
    yb = min(box1[3], box2[3])
    # compute the area of intersection rectangle
    interArea = max(0, xb - xa + 1) * max(0, yb - ya + 1)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    boxBArea = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [3]:
folder ='Combined'
df = pd.read_csv(f'../{folder}/PATHS.csv').drop('Unnamed: 0', axis =1)

zipped_paths = df[['filename']]

max_prop = 2000
max_pos = 5
max_neg = 5
input_dim = var.norm_dimension
total_pos = 0 
total_neg = 0
error_array = [] 
for idx in tqdm(range(len(df))): 
    row = df.iloc[idx]
    img_path = row.filename
    image = cv2.imread(f'../{folder}/{img_path}')
    try:
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        rects = ss.process() 
        prop_rect = [] 

        xMin = row.xmin
        xMax = row.xmax
        yMin = row.ymin
        yMax = row.ymax
        gt_box = ((xMin, yMin, xMax, yMax))
        gt_startx, gt_starty, gt_endx, gt_endy = gt_box
        for (x,y,w,h) in rects: 
            prop_rect.append((x,y,x+w, y+h))
        positive = 0 
        negative = 0 
        for rect in prop_rect[:max_prop]: 
            startx, starty, endx, endy = rect
            iou = get_iou(gt_box, rect)
            if iou > .7 and positive < max_pos: 
                roi = image[starty:endy, startx:endx]
                filename = f'{total_pos}.png'
                positive += 1
                total_pos += 1 
                roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
                cv2.imwrite(f'../FinalImages/Positive/{filename}', roi)
                continue

            fullOverlap = startx >= gt_startx
            fullOverlap = fullOverlap and starty >= gt_starty
            fullOverlap = fullOverlap and endx <= gt_endx
            fullOverlap = fullOverlap and endy <= gt_endy

            if not fullOverlap and iou < .05 and negative <= max_neg: 
                roi = image[starty:endy, startx: endx]
                filename = f'{total_neg}.png'
                negative += 1
                total_neg += 1
                roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
                cv2.imwrite(f'../FinalImages/Negative/{filename}', roi)
                continue 
    except: 
        error_array.append((img_path, image))
        continue
        
        
    
    



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6202/6202 [2:28:08<00:00,  1.43s/it]


In [4]:
print(error_array)

[('Pistol_107.jpg', None), ('Pistol_110.jpg', None), ('Pistol_111.jpg', None), ('Pistol_113.jpg', None), ('Pistol_1142_12.jpg', None), ('Pistol_117.jpg', None), ('Pistol_118.jpg', None), ('Pistol_1208_31.jpg', None), ('Pistol_121.jpg', None), ('Pistol_124.jpg', None), ('Pistol_125.jpg', None), ('Pistol_128.jpg', None), ('Pistol_1287_193.jpg', None), ('Pistol_129.jpg', None), ('Pistol_130.jpg', None), ('Pistol_132.jpg', None), ('Pistol_133.jpg', None), ('Pistol_134.jpg', None), ('Pistol_135.jpg', None), ('Pistol_138.jpg', None), ('Pistol_139.jpg', None), ('Pistol_140.jpg', None), ('Pistol_141.jpg', None), ('Pistol_142.jpg', None), ('Pistol_143.jpg', None), ('Pistol_144.jpg', None), ('Pistol_146.jpg', None), ('Pistol_147.jpg', None), ('Pistol_148.jpg', None), ('Pistol_149.jpg', None), ('Pistol_150.jpg', None), ('Pistol_151.jpg', None), ('Pistol_152.jpg', None), ('Pistol_153.jpg', None), ('Pistol_154.jpg', None), ('Pistol_155.jpg', None), ('Pistol_156.jpg', None), ('Pistol_157.jpg', None)

In [5]:
import pickle 

pickle.dump(error_array, open('../Pickles/error_array2.p', 'wb'))